# Import Libary

In [ ]:
import torch

print(f'Setup complete. Using torch {torch.__version__}')
print(f"{torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'}")

Setup complete. Using torch 2.0.1+cu117
NVIDIA A100-PCIE-40GB


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/


In [ ]:
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.153 🚀 Python-3.11.4 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100-PCIE-40GB, 40536MiB)
Setup complete ✅ (8 CPUs, 62.6 GB RAM, 45.4/48.9 GB disk)


In [ ]:
from ultralytics import RTDETR

## Connect WanDB

In [ ]:
!pip install wandb

  Using cached wandb-0.15.8-py3-none-any.whl (2.1 MB)


In [ ]:
%cd {HOME}
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/


wandb: Currently logged in as: boss39541. Use `wandb login --relogin` to force relogin


True

# Read Datasets from Kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                                   title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/countries-of-the-world-2023                          Global Country Information Dataset 2023             23KB  2023-07-08 20:37:33          10630        413  1.0              
nelgiriyewithana/global-youtube-statistics-2023                       Global YouTube Statistics 2023                      60KB  2023-07-28 15:36:38           2774        113  1.0              
joebeachcapital/global-earth-temperatures                             Global Earth Temperatures                           33KB  2023-08-09 06:21:51            577         27  1.0              
joebeachcapital/top-500-hollywood-m

In [ ]:
!kaggle datasets download -d mongkolboondamnoen/data-daji

100%|████████████████████████████████████████| 867M/867M [00:47<00:00, 20.9MB/s]
100%|████████████████████████████████████████| 867M/867M [00:47<00:00, 19.2MB/s]


## Unzip Datasets

In [ ]:
!unzip "data-daji.zip" -d "/root/datasets2"

Archive:  data-daji.zip
replace /root/datasets2/train/images/10000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


# Modeling

## Train Model

In [ ]:
model = RTDETR('rtdetr-l.pt')

params = {
    "task": "detect",
    "mode": "train",
    "model": "rtdetr-l.pt",
    "data": "root/cocogg.yaml",
    "epochs": 100,
    "batch": 30,
    "agnostic_nms": True,
    "cos_lr": True
}

wandb.init(project='MagiBoss30', config=params)
trainer = model.train(**params)
trainer.fit()

lr/pg0,▁▅██
lr/pg1,▁▅██
lr/pg2,▁▅██
metrics/mAP50(B),▁█▃
metrics/mAP50-95(B),▁█▃
metrics/precision(B),▁█▅
metrics/recall(B),▁█▃
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/cls_loss,█▁▁▁


Ultralytics YOLOv8.0.153 🚀 Python-3.11.4 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100-PCIE-40GB, 40536MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=root/cocogg.yaml, epochs=100, patience=50, batch=30, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=True, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=Fa

KeyboardInterrupt: 

# Test Model

In [ ]:
!unzip "/content/drive/MyDrive/NSC/Code Colab/test.zip" -d "/content"

args.yaml					   train_batch1.jpg
confusion_matrix_normalized.png			   train_batch22230.jpg
confusion_matrix.png				   train_batch22231.jpg
events.out.tfevents.1691893326.liver-0006.29023.0  train_batch22232.jpg
F1_curve.png					   train_batch2.jpg
labels_correlogram.jpg				   val_batch0_labels.jpg
labels.jpg					   val_batch0_pred.jpg
P_curve.png					   val_batch1_labels.jpg
PR_curve.png					   val_batch1_pred.jpg
R_curve.png					   val_batch2_labels.jpg
results.csv					   val_batch2_pred.jpg
results.png					   weights
train_batch0.jpg


In [ ]:
import torch
import cv2
from pathlib import Path
import numpy as np

/
Ultralytics YOLOv8.0.153 🚀 Python-3.11.4 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100-PCIE-40GB, 40536MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /root/datasets2/val/labels.cache... 874 images, 657 backgrounds, 0
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        874        248      0.655      0.575      0.566      0.335
                class1        874         68      0.533      0.688      0.502      0.303
                class2        874        128      0.703      0.469       0.54      0.254
                class3        874         52      0.729      0.568      0.656      0.447
Speed: 0.4ms preprocess, 3.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val12


In [ ]:
weights_path = '/content/drive/MyDrive/NSC/CodeColab/magi.pt'
model =  RTDETR(weights_path)

def detect_objects(image_path, model):
    results = model.predict(source=image_path, conf=0.25)
    boxes = results[0].boxes.xyxy[:, :4].cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    boxes_n = []
    for boxe in boxes:
      boxe = [float(i) for i in boxe]
      boxes_n.append(boxe)
    classes = [int(classe) for classe in classes]
    scores = [round(score, 4) for score in scores]
    return boxes_n, classes, scores

## Ans to .CSV

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/sample_submission.csv')
df

In [ ]:
b = []
c= []
s = []

for name in tqdm(df['id']):
  image_path = '/content/test/{0}'.format(name)
  boxes, classes, scores = detect_objects(image_path, model)
  b.append(boxes)
  c.append(classes)
  s.append(scores)

df['boxes'] = b
df['labels'] = c
df['scores'] = s
df

### Save .CSV

In [ ]:
df.to_csv('/content/submission08.csv', index=False)